# Setup

Mount Google Drive if in a Colab environment:



In [ ]:
import sys
if 'google' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    gdrive = True
else:
    gdrive = False

# Preparations

Prepare the Colab environment for imports. If using Google Drive the data should be in a directory called _'BIAI'_.

In [ ]:
FOLDER_HOME = '/content/drive/MyDrive/BIAI' if 'gdrive' in locals() and gdrive else '.'

sys.path.append(FOLDER_HOME)

Import the modules and define constants:

In [ ]:
from data import Loader, TestLogger, TrainLogger
from network import Network, Trainer, Tester, DEVICE

DATA_HOME = FOLDER_HOME + '/data'

LEARNING_RATE = 0.001
EPOCHS = 100

MODEL_OUTPUT_PATH = FOLDER_HOME + '/model.pt'
MODEL_INPUT_PATH = None

Define functions for training and testing:

In [ ]:
def train(net, trainer, train_data, train_logger, epoch):
  print(f'* Training epoch ({DEVICE}): {epoch}/{EPOCHS}...')
  
  net.train()
  train_logger.start(epoch)
  loss = trainer.train(net, train_data)
  train_logger.log_epoch(loss)
  
  print(f'* Finished training epoch: {epoch}/{EPOCHS}')

def test(net, tester, test_data, test_logger):
  print(f'* Testing...')
  
  net.eval()
  test_logger.start()
  correct, total = tester.test(net, test_data)
  test_logger.log_test_result(correct, total)
  
  print(f'Correct: {correct}/{total}, ({100 * correct / total:.2f}%)')

Create all the components:

In [ ]:
loader = Loader(DATA_HOME)
train_folder, train_data = loader.load_train()
test_folder, test_data = loader.load_test()
classes = train_folder.classes

net = None
if MODEL_INPUT_PATH:
  net = Network.load(MODEL_INPUT_PATH)
else:
  net = Network(len(classes), classes)

trainer = Trainer(LEARNING_RATE)
trainer.verbose = True
trainer.report_freq = 2
tester = Tester()

train_logger = TrainLogger(FOLDER_HOME + '/train.csv')
test_logger = TestLogger(FOLDER_HOME + '/test.csv')

# Main loop

In [ ]:
train(net, trainer, train_data, train_logger, 1)
net.save(MODEL_OUTPUT_PATH)
test(net, tester, test_data, test_logger)

train_logger.append = True
test_logger.append = True
for epoch in range(1, EPOCHS):
  train(net, trainer, train_data, train_logger, epoch + 1)
  net.save(MODEL_OUTPUT_PATH)
  test(net, tester, test_data, test_logger)

net.save(MODEL_OUTPUT_PATH)